In [2]:
!pip list #checking enviroment to xee if we are using sm2.0 or later

Keyring is skipped due to an exception: 'keyring.backends'
Package                              Version
------------------------------------ -----------------
aiobotocore                          2.4.0
aiohttp                              3.8.3
aioitertools                         0.11.0
aiosignal                            1.3.1
alabaster                            0.7.12
anaconda-client                      1.7.2
anaconda-project                     0.8.3
ansi2html                            1.8.0
anyio                                3.6.2
argh                                 0.26.2
argon2-cffi                          21.3.0
argon2-cffi-bindings                 21.2.0
asn1crypto                           1.3.0
astroid                              2.12.12
astropy                              4.0
async-timeout                        4.0.2
asynctest                            0.13.0
atomicwrites                         1.3.0
attrs                                22.1.0
autopep8         

In [4]:
#Importing libraries
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [5]:
#reading data using pandas
fuel_economy_df=pd.read_csv('FuelEconomy.csv')

In [6]:
#view the dataframe
fuel_economy_df

,Horse Power,Fuel Economy (MPG)
0,118.770799,29.344195
1,176.326567,24.695934
2,219.262465,23.952010
3,187.310009,23.384546
4,218.594340,23.426739
...,...,...
95,162.810542,27.418661
96,266.869640,15.989945
97,243.831211,19.253375
98,140.959803,29.515593


In [7]:
fuel_economy_df.head

<bound method NDFrame.head of     Horse Power  Fuel Economy (MPG)
0    118.770799           29.344195
1    176.326567           24.695934
2    219.262465           23.952010
3    187.310009           23.384546
4    218.594340           23.426739
..          ...                 ...
95   162.810542           27.418661
96   266.869640           15.989945
97   243.831211           19.253375
98   140.959803           29.515593
99   184.123053           25.196097

[100 rows x 2 columns]>

In [9]:
#Seprate the data into input X and output Y
x=fuel_economy_df[['Horse Power']]
y=fuel_economy_df[['Fuel Economy (MPG)']]

In [10]:
x.shape #shape of input

(100, 1)

In [11]:
y.shape

(100, 1)

In [12]:
#Convert data into float32
X=np.array(x).astype('float32')
Y=np.array(y).astype('float32')

In [13]:
#only take numerical variable
X

array([[118.7708  ],
       [176.32657 ],
       [219.26247 ],
       [187.31001 ],
       [218.59435 ],
       [175.8381  ],
       [271.44162 ],
       [294.4259  ],
       [126.211006],
       [163.35034 ],
       [321.84076 ],
       [120.48424 ],
       [155.41537 ],
       [191.71481 ],
       [211.72911 ],
       [259.1832  ],
       [236.57173 ],
       [191.09897 ],
       [123.8857  ],
       [136.30646 ],
       [212.73895 ],
       [232.44995 ],
       [122.04016 ],
       [140.43231 ],
       [243.34784 ],
       [178.43028 ],
       [258.42422 ],
       [231.8549  ],
       [ 91.44026 ],
       [119.53666 ],
       [350.      ],
       [175.97922 ],
       [312.49222 ],
       [282.60425 ],
       [138.26474 ],
       [301.05417 ],
       [227.8735  ],
       [ 50.      ],
       [152.34785 ],
       [252.55238 ],
       [218.10709 ],
       [172.64366 ],
       [243.73724 ],
       [186.04004 ],
       [227.67047 ],
       [132.66957 ],
       [196.85954 ],
       [244.3

In [15]:
# Splitting the data and testing using sklearn library
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)

In [17]:
# Train a linear model using AWS Sagemaker
import sagemaker
import boto3 #It is AWS SDk for python.It facilates the libraries and method of python

In [19]:
# create bucket
bucket='aws.ml.engineer5' #bucket need to created forhead
prefix='linear_learner_fuel_economy' #subfolder within bucket

In [20]:
role=sagemaker.get_execution_role() #making role to access the bucket 
print(role)

arn:aws:iam::451246788348:role/service-role/AmazonSageMaker-ExecutionRole-20221121T170854


In [21]:
X_train.shape

(80, 1)

In [22]:
Y_train=Y_train[:,0]

In [23]:
Y_train.shape

(80,)

In [24]:
import io #This will allow for various I/O operation
import sagemaker.amazon.common as smac #sagemaker common library 


In [27]:
#COnverting the data into numpy array format to Record IO format
#this is the format required by sagemaker linear learner
buf=io.BytesIO() #create an in-memory byte array
smac.write_numpy_to_dense_tensor(buf,X_train,Y_train)
buf.seek(0)

0

In [28]:
#Code to upload RecordIO data to S3
#key refers to name of the file
key='linear-train-data'

In [31]:
# The following code uploads the data in record-io format to S3 bucket to be accessed later for training
import os
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)
 
# Let's print out the training data location in s3
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))

uploaded training data location: s3://aws.ml.engineer5/linear_learner_fuel_economy/train/linear-train-data


In [32]:
X_test.shape

(20, 1)

In [33]:
Y_test.shape


(20, 1)

In [34]:
#make sure that the target label is a vector
Y_test=Y_test[:,0]

In [35]:
buf=io.BytesIO() #create an in-memory byte array
smac.write_numpy_to_dense_tensor(buf,X_test,Y_test)
buf.seek(0)

0

In [36]:
# The following code uploads the data in record-io format to S3 bucket to be accessed later for training
import os
key='linear-test-data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test', key)).upload_fileobj(buf)
 
# Let's print out the training data location in s3
s3_test_data = 's3://{}/{}/test/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_test_data))

uploaded training data location: s3://aws.ml.engineer5/linear_learner_fuel_economy/test/linear-test-data
